# L’analyse en composantes principales (ACP)

Le principe derrière l’ACP (ou PCA en anglais pour *principal component analysis*) serait de résumer l’information contenue dans une base de données grâce à des variables synthétiques appelées **composantes principales**. Grâce à des méthodes statistiques, les variables retenues pour l’analyse sont combinées entre elles afin de mettre en évidence des traits qui rassemblent le maximum de la variance des données.

Nous parlons bien d’un maximum sans garantir l’intégralité. Toute compression implique inévitablement une perte de l’information, mais elle se fait au profit d’un gain de performance et d’acquisition des données.

Si les outils informatiques permettent d’obtenir rapidement une ACP, il est important d’en comprendre les étapes afin, peut-être, d’améliorer les paramètres de l’algorithme. Nous allons dans un premier temps effectuer l’analyse à la main avant de mobiliser des outils informatiques.

Commençons par charger les librairies nécessaires :

In [ ]:
import pandas as pd

## Une ACP sur l’indicateur du vivre mieux

### Présentation de l’enquête

Chargeons une enquête, limitée aux données concernant les femmes des pays de l’OCDE et augmentée de la Russie et de l’Afrique du Sud, sur l’indicateur du vivre de mieux et affichons les premières lignes afin de prendre connaissance des informations enregistrées :

In [ ]:
df = pd.read_csv("./data/better-life-index-women-2021.csv", delimiter=",")

df.head()

L’enquête comporte un certain nombre de variables numériques expliquées ci-dessous :

|Variable|Signification|
|:-:|-|
|*code du pays*|Pays|
|*country*|Pays|
|*PS_FSAFEN*|Se sentir en sécurité quand on marche seul la nuit|
|*JE_EMPL*|Taux d’emploi|
|*JE_LTUR*|Taux de chômage de longue durée|
|*SC_SNTWS*|Qualité du réseau social|
|*ES_EDUA*|Niveau d’instruction|
|*ES_STCS*|Compétences des élèves|
|*ES_EDUEX*|Années de scolarité|
|*EQ_WATER*|Qualité de l’eau|
|*HS_LEB*|Espérance de vie|
|*HS_SFRH*|Auto-évaluation de l’état de santé|
|*SW_LIFS*|Satisfaction à l’égard de la vie|
|*PS_REPH*|Taux d’homicides|
|*WL_EWLH*|Horaires de travail lourds|

### Centrer et réduire les données

On le remarque aisément, toutes les variables ne sont pas codées sur la même échelle : parfois nous avons des pourcentages, d’autres un nombre d’années, ou encore une évaluation de 0 à 10. Afin de limiter l’influence d’une variable sur l’autre, le premier réflexe est de centrer les valeurs autour de la moyenne voire de les réduire. Pour centrer une variable, il suffit d’appliquer la formule $x = X - \mu$ et, pour la réduire, il faut ensuite la diviser par l’écart-type :

$$
x = \frac{X - \mu}{\sigma}
$$

Le principal avantage d’une variable centrée est qu’elle dispose d’une espérance nulle (la moyenne de ses observations sera proche de zéro). Si elle est en plus réduite, elle deviendra indépendante de l’unité et son écart-type comme sa variance égaleront 1.

Préparons une copie des données numériques en éliminant les deux observations qui comportent des valeurs vides (Slovénie et Afrique du Sud) et centrons la première variable :

In [ ]:
# select only numerical columns
features = df.columns
features = features.drop(["code", "country"])

# drop rows with NA values
data = df.copy().dropna()

# centering first feature
data.PS_FSAFEN = data.PS_FSAFEN - data.PS_FSAFEN.mean()

La moyenne de la variable `PS_FSAFEN` devrait maintenant avoisiner 0 :

In [ ]:
data.PS_FSAFEN.mean().round(2)

Si ensuite nous la réduisons, son écart-type est fixé autour de 1 :

In [ ]:
# scaling first feature
data.PS_FSAFEN = data.PS_FSAFEN / data.PS_FSAFEN.std()

data.PS_FSAFEN.std().round(2)

Il reste à appliquer le même traitement à toutes les colonnes :

In [ ]:
# centering and scaling each numerical feature
for feature in data[features]:
    data[feature] = data[feature].transform(lambda x: (x - x.mean()) / x.std())

Un coup d’œil rapide à nos données nous prouve que la transformation a bien été appliquée :

In [ ]:
data[features].describe().round(2)